<center>
<img src="https://i.pinimg.com/564x/e7/48/d2/e748d2bdcc1132ec0d427f6db7e833d6.jpg" width="600"> 

# Предсказываем количество лайков на Яндекс Музыке
</center>

**ФИО:** Колесникова Анастасия Ильинична (241), Скуртул Надежда Анатольевна (257)

# Шаг -1. Выбираем тему

Тема уже была выбрана и подробно описана в форме, а также подробный порядок действий будет затрагиваться ниже, но общую суть проекта мы напомним здесь.

__Задача проекта:__ На основе кагловского датасета (https://www.kaggle.com/datasets/pieca111/music-artists-popularity), информации, собранной с API Яндекс музыки и английской википедии, найти и проверить интересную гипотезу, а также обучить модель предсказывать количество лайков на Яндекс Музыке у исполнителя.

Импортируем необходимые библиотеки:

In [1]:
import numpy as np
import pandas as pd

import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from tqdm.notebook import tqdm
from pprint import pprint

In [3]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import bs4
import re
import math

In [4]:
# для красивых графиков

%config InlineBackend.figure_format = 'retina'

sns.set(style='darkgrid', palette='deep')

plt.rcParams['font.size'] = 16
plt.rcParams['savefig.format'] = 'pdf'

Теперь мы готовы начать работу.

<center>
<img src="https://i.pinimg.com/564x/ef/ce/fa/efcefa78296109467a6b1576c89d5a58.jpg" width="400"> 

</center>

# Шаг 0. Подгрузим готовый датасет с Kaggle и обработаем его

Подгрузим таблицу с Kaggle (ссылку смотри выше, также сам файл с датасетом прикрепим на ГитХабе отдельно). Удалим ненужную колонку 'mbid', так как она содержит id исполнителей с сайта musicbrainz, а мы будем делать предсказания на основе приложения Яндекс Музыка, а также столбцы 'artist_lastfm', 'country_lastfm', 'tags_mb', 'tags_lastfm', 'ambiguous_artist', так как они не представляют ценности при обучении нашей модели. Также удалим повторяющиеся группы.

In [25]:
df = pd.read_csv('artists.csv', sep=',')
df = df.drop(['mbid', 'artist_lastfm', 'country_lastfm', 'tags_mb', 'tags_lastfm', 'ambiguous_artist'], axis=1)
df.drop_duplicates(subset='artist_mb', inplace=True)
print(df.shape)
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(1352997, 4)


,artist_mb,country_mb,listeners_lastfm,scrobbles_lastfm
0,Coldplay,United Kingdom,5381567.0,360111850.0
1,Radiohead,United Kingdom,4732528.0,499548797.0
2,Red Hot Chili Peppers,United States,4620835.0,293784041.0
3,Rihanna,United States,4558193.0,199248986.0
4,Eminem,United States,4517997.0,199507511.0


У нас огромное количество наблюдений (1352997, как видно из размера массива), поэтому работу с пропусками мы можем позволить себе ещё на этапе сбора данных. Удалим строки (исполнителей и группы) с хотя бы одной пустой ячейкой.

In [26]:
df = df.dropna()
print(df.shape)
df.head()

(420884, 4)


,artist_mb,country_mb,listeners_lastfm,scrobbles_lastfm
0,Coldplay,United Kingdom,5381567.0,360111850.0
1,Radiohead,United Kingdom,4732528.0,499548797.0
2,Red Hot Chili Peppers,United States,4620835.0,293784041.0
3,Rihanna,United States,4558193.0,199248986.0
4,Eminem,United States,4517997.0,199507511.0


В датафрейме мы с помощью колонки 'listeners_lastfm' отобрали из 420884 исполнителей, оставшихся после удаления строк с путями ячеками, 5000 самых популярных, с которыми мы будем работать на Яндекс Музыке (для облегчения работы с циклами в промежуточном вариенте использовали 1500, но в финальной версии работы оставим 5000). Мы можем выбрать популярных исполнителей так вместо того, чтобы выбрать самые популярные группы по данным Яндекс Музыки, так как такая выборка позволит нам обучить модель на группах, которые, возможно, не так популярны на российской платформе. Соответственно модель будет обучена и не не самых популярных в России группах и сможет лучше прогнозировать популярность исполнителя.

In [28]:
df = df.nlargest(5000, 'listeners_lastfm')
print(df.shape)
df.head()

(5000, 4)


,artist_mb,country_mb,listeners_lastfm,scrobbles_lastfm
0,Coldplay,United Kingdom,5381567.0,360111850.0
1,Radiohead,United Kingdom,4732528.0,499548797.0
2,Red Hot Chili Peppers,United States,4620835.0,293784041.0
3,Rihanna,United States,4558193.0,199248986.0
4,Eminem,United States,4517997.0,199507511.0


У нас есть готовый список исполнителей, с которыми мы можем работать. Дальнейший порядок сбора данных:

1. С помощью Яндекс API подгрузим количество лайков (на основе которых и будем определять популярность исполнителя, так как на Яндекс Музыке указано только количество прослушиваний за последний месяц, а не накопительный эффект), количество треков и количество альбомов у исполнителя. 

2. Обратимся к англоязычной Википедии, спарсим с неё данные о каждом исполнителе: количество лет работы исполнителя или группы, создадим переменные, закодированные как 0 или 1, в зависимости от того, продолжает ли работать исполнитель на текущий момент, прерывала ли группа своё существование / исполнитель свою музыкальную карьеру.

Таким образом.

Что нужно достать с Яндекс API:
- количество лайков у исполнителя (наш таргет)
- количество треков
- количество альбомов

Что нужно достать с вики:
- сколько работает группа / исполнитель
- работает ли до сих пор (бинарная переменная)

__Да начнётся мясо.__

<center>
<img src="https://preview.redd.it/hcz8vstwwbx91.jpg?width=750&format=pjpg&auto=webp&v=enabled&s=edbaeae53873c98b34cabff57e0cbef7951add48" width="400"> 

</center>

# Шаг 1. Работаем с Яндекс API.

1.1. Установим Яндекс API и создадим экземпляр клиента. Мы инициализируем синхронного клиента, так как у нас нет необходимости в обработке большого количества запросов и не требуется высокая производительность. Также подгрузим наш токен.

In [10]:
pip install -U yandex-music

In [36]:
from yandex_music import Client

client = Client('здесь должен быть ваш токен :)').init()

Здесь (в окне ниже) мы пытались разобраться, как достать необходимые нам признаки с помощью API. Здесь не представлены все наши страдания, но вы можете прочувствовать их, окунувшись в документацию, представленную по ссылке: https://yandex-music.readthedocs.io/en/main/index.html. Половина функций по какой-то причине не работает, а официальной постоянно обновляемой документации от Yandex, к сожалению, нет. 

Ниже представлен закоменченный шаг, когда мы всё-таки поняли, как добраться до информации исполнителей по API. 

In [107]:
# a = client.search("Rihanna")
# nirv_id = a.best.to_dict()['result']['id']

# # pprint([key for key in client.artistsBriefInfo(nirv_id).to_dict().keys()])
# # pprint(client.artistsBriefInfo(nirv_id).to_dict()['artist'])

# pprint(client.artistsBriefInfo(nirv_id).to_dict()['artist']['likes_count'])
# pprint(client.artistsBriefInfo(nirv_id).to_dict()['artist']['genres'])
# pprint(client.artistsBriefInfo(nirv_id).to_dict()['artist']['en_wikipedia_link'])

Дальше всю необходимую нам информацию мы получили для 5000 исполнителей с помощью цикла. Его выполнение занимает около 40 минут, возможно, мы постараемся усовершенствовать его в будущем.

In [13]:
artist_names = df['artist_mb'].tolist()

In [14]:
artist_track_counts = {}

for i in tqdm(artist_names):
    try:
        artist = client.search(i)
        artist_id = artist.best.to_dict()['result']['id']
        likes = client.artistsBriefInfo(artist_id).to_dict()['artist']['likes_count']
        tracks = client.artistsBriefInfo(artist_id).to_dict()['artist']['counts']['tracks']
        albums = client.artistsBriefInfo(artist_id).to_dict()['artist']['counts']['direct_albums']
        genres = client.artistsBriefInfo(artist_id).to_dict()['artist']['genres']
        wiki = client.artistsBriefInfo(artist_id).to_dict()['artist']['en_wikipedia_link']
        artist_track_counts[i] = likes, tracks, albums, genres, wiki
    except:
        print(f"Исполнитель {i} не найден")

  0%|          | 0/5000 [00:00<?, ?it/s]

Исполнитель Air не найден
Исполнитель Fleetwood Mac не найден
Исполнитель Stevie Wonder не найден
Исполнитель Dire Straits не найден
Исполнитель Sia не найден
Исполнитель Deep Purple не найден
Исполнитель The Kinks не найден
Исполнитель Jennifer Lopez не найден
Исполнитель The Notorious B.I.G. не найден
Исполнитель Phoenix не найден
Исполнитель CAKE не найден
Исполнитель Stone Temple Pilots не найден
Исполнитель Motörhead не найден
Исполнитель The Fratellis не найден
Исполнитель Spoon не найден
Исполнитель EELS не найден
Исполнитель Neutral Milk Hotel не найден
Исполнитель Scissor Sisters не найден
Исполнитель Stars не найден
Исполнитель Glee Cast не найден
Исполнитель Noah and the Whale не найден
Исполнитель Flyleaf не найден
Исполнитель Nouvelle Vague не найден
Исполнитель AWOLNATION не найден
Исполнитель Emilíana Torrini не найден
Исполнитель Hot Hot Heat не найден
Исполнитель Erasure не найден
Исполнитель Anti‐Flag не найден
Исполнитель Poison не найден
Исполнитель +44 не найден
Ис

In [108]:
# print(artist_track_counts)

Теперь нашей задачей было оформить новые данные из словаря в датафрейм. Для этого пришлось его немного повертеть.

In [15]:
df_yandex = pd.DataFrame(artist_track_counts)
df_yandex.head()

,Coldplay,Radiohead,Red Hot Chili Peppers,Rihanna,Eminem,The Killers,Kanye West,Nirvana,Muse,Queen,...,横田進,Hopsin,Nellie McKay,Lyn Collins,Saigon,Kwabs,Astral Projection,Diablo Swing Orchestra,Despised Icon,Samuel E. Wright
0,1305445,387180,2155459,1700983,5278564,376882,2030553,3584013,1039519,4498057,...,1,3483,118,109,82,1207967,20533,15532,3009,34
1,186,200,266,268,396,151,386,165,149,297,...,10,163,87,43,298,28,154,60,61,24
2,60,42,32,57,37,25,51,15,35,38,...,1,41,5,4,42,8,23,5,3,0
3,[rock],[indie],[rock],"[pop, dance, rnb]",[foreignrap],"[alternative, rock, indie]",[foreignrap],"[rock, alternative]","[alternative, rock]","[rock, hardrock, films]",...,[trance],[rap],"[pop, jazz]","[rnb, electronics]","[rap, pop, african]",[soul],[trance],[progmetal],[metalcoregenre],[]
4,http://en.wikipedia.org/wiki/Coldplay,http://en.wikipedia.org/wiki/Radiohead,http://en.wikipedia.org/wiki/Red Hot Chili Pep...,http://en.wikipedia.org/wiki/Rihanna,http://en.wikipedia.org/wiki/Eminem,http://en.wikipedia.org/wiki/The Killers,http://en.wikipedia.org/wiki/Kanye West,http://en.wikipedia.org/wiki/Nirvana (band),http://en.wikipedia.org/wiki/Muse (band),http://en.wikipedia.org/wiki/Queen (band),...,None,http://en.wikipedia.org/wiki/Hopsin,None,http://en.wikipedia.org/wiki/Lyn Collins,http://en.wikipedia.org/wiki/Saigon (rapper),http://en.wikipedia.org/wiki/Kwabs,http://en.wikipedia.org/wiki/Astral Projection...,http://en.wikipedia.org/wiki/Diablo Swing Orch...,http://en.wikipedia.org/wiki/Despised Icon,http://en.wikipedia.org/wiki/Samuel E. Wright


In [16]:
columns_names = ['Количество лайков', 'Количество треков', 'Количество альбомов', 'Жанры', 'Страница Википедии']

In [17]:
df_yandex = df_yandex.T.set_axis(columns_names, axis=1, inplace=False)
df_yandex.head(10)

,Количество лайков,Количество треков,Количество альбомов,Жанры,Страница Википедии
Coldplay,1305445,186,60,[rock],http://en.wikipedia.org/wiki/Coldplay
Radiohead,387180,200,42,[indie],http://en.wikipedia.org/wiki/Radiohead
Red Hot Chili Peppers,2155459,266,32,[rock],http://en.wikipedia.org/wiki/Red Hot Chili Pep...
Rihanna,1700983,268,57,"[pop, dance, rnb]",http://en.wikipedia.org/wiki/Rihanna
Eminem,5278564,396,37,[foreignrap],http://en.wikipedia.org/wiki/Eminem
The Killers,376882,151,25,"[alternative, rock, indie]",http://en.wikipedia.org/wiki/The Killers
Kanye West,2030553,386,51,[foreignrap],http://en.wikipedia.org/wiki/Kanye West
Nirvana,3584013,165,15,"[rock, alternative]",http://en.wikipedia.org/wiki/Nirvana (band)
Muse,1039519,149,35,"[alternative, rock]",http://en.wikipedia.org/wiki/Muse (band)
Queen,4498057,297,38,"[rock, hardrock, films]",http://en.wikipedia.org/wiki/Queen (band)


In [18]:
df_yandex.to_csv('Yandex_Dataframe_5000.csv')

Чтобы вам не пришлось самостоятельно проделывать 40-минутный цикл, сохранили датафрейм в отдельный файл csv.

А дальше мы начали страдать...

<center>
<img src="https://i.pinimg.com/564x/f7/b5/3e/f7b53edd2b07a413832ddc0da1afdf90.jpg" width="400"> 

</center>

# Шаг 2. Прасим англоязычную Википедию

Работа продолжается, и теперь мне впадлу проделывать 40-минутный цикл, поэтому импортирую из сохранённого файла.

In [5]:
df_yandex = pd.read_csv('Yandex_Dataframe_5000.csv', sep=',')
df_yandex.rename(columns={'Unnamed: 0': 'artist_mb'}, inplace=True)
df_yandex.set_index('artist_mb', inplace=True)
print(df_yandex.shape)
df_yandex.head()

(4862, 5)


,Количество лайков,Количество треков,Количество альбомов,Жанры,Страница Википедии
artist_mb,,,,,
Coldplay,1305445,186,60,['rock'],http://en.wikipedia.org/wiki/Coldplay
Radiohead,387180,200,42,['indie'],http://en.wikipedia.org/wiki/Radiohead
Red Hot Chili Peppers,2155459,266,32,['rock'],http://en.wikipedia.org/wiki/Red Hot Chili Pep...
Rihanna,1700983,268,57,"['pop', 'dance', 'rnb']",http://en.wikipedia.org/wiki/Rihanna
Eminem,5278564,396,37,['foreignrap'],http://en.wikipedia.org/wiki/Eminem


Мы собирали года активности исполнителей, затем пытались обработать их (убрать ссылки в квадратых скобках, заменить "на настоящий момент" на год 2023, а после вычислить разность, чтобы получить количество лет активности исполнителя). Здесь можете видеть некоторые функции для работы.

В ячейке ниже пытались понять, как парсить данные с Википедии: выяснили, по каким тегам лежит нужная нам информация с годами функционирования исполнителей, указание каких классов и другой дополнительной информации помогает к ним перейти.

In [9]:
url = "https://en.wikipedia.org/wiki/Eminem"
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find('table', {'class': 'infobox biography vcard'})

# rows = table.find_all('tr')
# for row in rows:
#     cols = row.find_all('th')
#     cols = [ele.text.strip() for ele in cols]
#     print(cols)

print(table.find('th', text="Years\xa0active"))

years_active_row = table.find('th', text='Years\xa0active').parent
years_active = years_active_row.find('td').text.strip()

print(f"Eminem's years active: {years_active}")

<th class="infobox-label" scope="row">Years active</th>
Eminem's years active: 1988–present[1]


Задали функции, необходимые для обработки данных.

In [16]:
def get_intervals(raw_intervals):
    intervals = []
    # split by '-'
    left_rights = re.findall(r'\S+–\S+', raw_intervals) # обратите внимание: там не обычное тире, а какая-то хрень
    for left_right in left_rights:
        parts = left_right.split('–') # обратите внимание: там не обычное тире, а какая-то хрень
        clean_parts = tuple(re.findall(r'(?:\d{4}|present)', part)[0] for part in parts)
        clean_parts_without_present = tuple(2023 if part == 'present' else int(part) for part in clean_parts)
        intervals.append(clean_parts_without_present)
    return intervals

def parse_years_text(s):
    # check hiaustes
    main_and_hiatuses = s.split('hiatus')
    main = main_and_hiatuses[0]
    hiastuses = ''
    if len(main_and_hiatuses) > 1:
        hiastuses = main_and_hiatuses[1]

    main_intervals = get_intervals(main)
    hiastuses_intervals = get_intervals(hiastuses)
        
    return main_intervals, hiastuses_intervals

def get_band_activity_duration(main_intervals, hiastuses_intervals):
    duration = 0
    for interval in main_intervals:
        duration += interval[1] - interval[0]
    for interval in hiastuses_intervals:
        duration -= interval[1] - interval[0]
    return duration

В ячейке ниже представлен код, который позволяет непосредственно спарсить годы активности с англоязычной Википедии.

In [27]:
big_lst = []

for i in tqdm(range(df_yandex.shape[0])):

    try:
        url = df_yandex.iloc[i, 4]
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'class': 'infobox biography vcard'})

        if soup.find('table', {'class': 'infobox biography vcard'}) is None:
            table = soup.find('table', {'class': 'infobox vcard plainlist'})
            years_active_row = table.find('th', text='Years active').parent

        elif table.find('th', text='Years\xa0active') is None:
            years_active_row = table.find('th', text='Years active').parent

        else:
            table = soup.find('table', {'class': 'infobox biography vcard'})
            years_active_row = table.find('th', text='Years\xa0active').parent  

        years = years_active_row.find('td')
        big_lst.append(years)
        
    except:
        big_lst.append(None)

  0%|          | 0/4862 [00:00<?, ?it/s]

In [28]:
df_parced = pd.DataFrame({'artist_mb': df_yandex.index,
                   'parsed_data': big_lst})
df_parced.head()

,artist_mb,parsed_data
0,Coldplay,[1997–present]
1,Radiohead,[1985–present]
2,Red Hot Chili Peppers,"[1982, [ (, [1982], )], –present]"
3,Rihanna,[2003–present]
4,Eminem,"[1988–present, [[[1]]]]"


In [29]:
df_parced.to_csv('big_lst_5000.csv')

In [6]:
df_parsed = pd.read_csv('big_lst_5000.csv', sep=',')
df_parsed.rename(columns={'parsed_data': 'parsed_text'}, inplace=True)
df_parsed = df_parsed.drop(['Unnamed: 0'], axis=1)
print(df_parsed.shape)
df_parsed.head()

(4862, 2)


,artist_mb,parsed_text
0,Coldplay,"<td class=""infobox-data"">1997–present</td>"
1,Radiohead,"<td class=""infobox-data"">1985–present</td>"
2,Red Hot Chili Peppers,"<td class=""infobox-data"">1982<span style=""disp..."
3,Rihanna,"<td class=""infobox-data"">2003–present</td>"
4,Eminem,"<td class=""infobox-data"">1988–present<sup clas..."


In [17]:
big_lst_years = []

for i in tqdm(range(df_parsed.shape[0])):
    
    try:        
        years = bs4.BeautifulSoup(df_parsed.iloc[i, 1])
        main, hiatuses = parse_years_text(years.text)
        duration = get_band_activity_duration(main, hiatuses)
        big_lst_years.append(duration)
    
    except:
        big_lst_years.append(None)

  0%|          | 0/4862 [00:00<?, ?it/s]

In [18]:
len(np.where(np.array(big_lst_years) == None)[0].tolist())

447

Количество данных позволяет нам удалить исполнителей, у которых слишком сложная цепочка парсинга данных, что мы и сделаем в предварительной обработке данных (шаг 3).

# Шаг 3. Обрабатываем данные перед их анализом

Задача: _Проанализируйте столбцы. Что в них содержится? Опишите переменные. Есть ли в данных пропуски? Обработайте пропуски. Если в какой-то колонке пропусков большинство, то ее можно удалить. Если их не очень много, можно заполнить их медианой/средним или чем-нибудь еще, напишите почему вы выбрали тот или иной вариант._

Для начала добавим наш лист с годами активности групп и исполнителей в df_yandex.

### Обработаем пропуски

In [19]:
df_yandex['Годы активности'] = big_lst_years
print(df_yandex.shape)
df_yandex.head()

(4862, 6)


,Количество лайков,Количество треков,Количество альбомов,Жанры,Страница Википедии,Годы активности
artist_mb,,,,,,
Coldplay,1305445,186,60,['rock'],http://en.wikipedia.org/wiki/Coldplay,26.0
Radiohead,387180,200,42,['indie'],http://en.wikipedia.org/wiki/Radiohead,38.0
Red Hot Chili Peppers,2155459,266,32,['rock'],http://en.wikipedia.org/wiki/Red Hot Chili Pep...,41.0
Rihanna,1700983,268,57,"['pop', 'dance', 'rnb']",http://en.wikipedia.org/wiki/Rihanna,20.0
Eminem,5278564,396,37,['foreignrap'],http://en.wikipedia.org/wiki/Eminem,35.0


Удалим исполнителей, для которых года активности не были спарсены, и поменяем тип данных колонки 'Года активности' с float на int. Также удалим колонку 'Страница Википедии', так как для анализа она не представляется полезной.

In [21]:
df_yandex = df_yandex.dropna()
df_yandex['Годы активности'] = df_yandex['Годы активности'].astype(int)
df_yandex = df_yandex.drop('Страница Википедии', axis=1)
print(df_yandex.shape)
df_yandex.head()

(4415, 5)


,Количество лайков,Количество треков,Количество альбомов,Жанры,Годы активности
artist_mb,,,,,
Coldplay,1305445,186,60,['rock'],26
Radiohead,387180,200,42,['indie'],38
Red Hot Chili Peppers,2155459,266,32,['rock'],41
Rihanna,1700983,268,57,"['pop', 'dance', 'rnb']",20
Eminem,5278564,396,37,['foreignrap'],35


Осталось 1412 наблюдений, что является достаточно большой выборкой для нашей работы. Таким образом, мы __обработали пропуски__, так как в данной задаче вставлять среднее/медиану или заполнять пропуски нулями было бы некорректно.

### Сведём готовые данные с kaggle и спарсенные данные

Сделаем исполнителей и групп отдельной колонкой (из индексов).

In [22]:
df_yandex.reset_index(inplace=True)
df_yandex.head()

,artist_mb,Количество лайков,Количество треков,Количество альбомов,Жанры,Годы активности
0,Coldplay,1305445,186,60,['rock'],26
1,Radiohead,387180,200,42,['indie'],38
2,Red Hot Chili Peppers,2155459,266,32,['rock'],41
3,Rihanna,1700983,268,57,"['pop', 'dance', 'rnb']",20
4,Eminem,5278564,396,37,['foreignrap'],35


Переименуем колонку, как в исходном датафрейме из kaggle.

In [23]:
df_yandex.rename(columns={'index': 'artist_mb'}, inplace=True)
df_yandex.head()

,artist_mb,Количество лайков,Количество треков,Количество альбомов,Жанры,Годы активности
0,Coldplay,1305445,186,60,['rock'],26
1,Radiohead,387180,200,42,['indie'],38
2,Red Hot Chili Peppers,2155459,266,32,['rock'],41
3,Rihanna,1700983,268,57,"['pop', 'dance', 'rnb']",20
4,Eminem,5278564,396,37,['foreignrap'],35


Теперь мы можем смёрджить наши два датафрейма с готовыми данными и теми, которым мы спарсили с Яндекс Музыки и Википедии.

In [29]:
df_res = df.merge(df_yandex, on='artist_mb')
print(df_res.shape)
df_res.head()

(4415, 9)


,artist_mb,country_mb,listeners_lastfm,scrobbles_lastfm,Количество лайков,Количество треков,Количество альбомов,Жанры,Годы активности
0,Coldplay,United Kingdom,5381567.0,360111850.0,1305445,186,60,['rock'],26
1,Radiohead,United Kingdom,4732528.0,499548797.0,387180,200,42,['indie'],38
2,Red Hot Chili Peppers,United States,4620835.0,293784041.0,2155459,266,32,['rock'],41
3,Rihanna,United States,4558193.0,199248986.0,1700983,268,57,"['pop', 'dance', 'rnb']",20
4,Eminem,United States,4517997.0,199507511.0,5278564,396,37,['foreignrap'],35


Переведём названия (переименуем) колонок нашего датафрейма на один язык для удобства и упрощённого восприятия данных.

In [33]:
df_res.rename(columns={'artist_mb': 'artist'}, inplace=True)
df_res.rename(columns={'country_mb': 'country'}, inplace=True)
df_res.rename(columns={'Количество лайков': 'likes'}, inplace=True)
df_res.rename(columns={'Количество треков': 'tracks'}, inplace=True)
df_res.rename(columns={'Количество альбомов': 'albums'}, inplace=True)
df_res.rename(columns={'Жанры': 'genres'}, inplace=True)
df_res.rename(columns={'Годы активности': 'years_active'}, inplace=True)
df_res.head()

,artist,country,listeners_lastfm,scrobbles_lastfm,likes,tracks,albums,genres,years_active
0,Coldplay,United Kingdom,5381567.0,360111850.0,1305445,186,60,['rock'],26
1,Radiohead,United Kingdom,4732528.0,499548797.0,387180,200,42,['indie'],38
2,Red Hot Chili Peppers,United States,4620835.0,293784041.0,2155459,266,32,['rock'],41
3,Rihanna,United States,4558193.0,199248986.0,1700983,268,57,"['pop', 'dance', 'rnb']",20
4,Eminem,United States,4517997.0,199507511.0,5278564,396,37,['foreignrap'],35


Итоговый для промежуточного этапа датасет сохранили в отдельном датафрейме. 

In [34]:
df_res.to_csv('Interim_Project_5000.csv')

### Опишем переменные

In [32]:
df_res.dtypes

artist               object
country              object
listeners_lastfm    float64
scrobbles_lastfm    float64
likes                 int64
tracks                int64
albums                int64
genres               object
years_active          int32
dtype: object

По типу данных мы можем увидеть, что в датафрейме есть перменные со следующими соответствующими им признаками:

- числовые (5 шт.): listeners_lastfm (количество слушателей на Last.fm), scrobbles_lastfm (количество прослушиваний на Last.fm), likes (количество лайков), tracks (количество треков), albums (количество альбомов), years_active (годы активности)

- категориальные (2 шт.): country (страна), genres (жанры)